In [1]:
import pandas as pd
import numpy as np
import os
import h5py

In [2]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install osmnx

Note: you may need to restart the kernel to use updated packages.


In [4]:
# geography
import geopandas as gpd
import osmnx as ox
import csv
import ast
from geopy.geocoders import Nominatim
from geopy.point import Point

In [80]:
df = pd.read_csv('/Users/katiecason/Desktop/207_folder/w207_finalproject/tweets_slightly_cleaned.csv')

/Users/katiecason/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [81]:
df = df[df['geo'].notna()]

In [82]:
df.geo = df.geo.apply(lambda x: ast.literal_eval(x))

In [83]:
df.geo = df.geo.apply(lambda x: x['coordinates'])

In [84]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [85]:
#manually filtering latitude and longitude to find locations

#df = df[df['coordinates'].notna()]
#data = df['coordinates'].str.split(',', n=1, expand = True)
#df['longitude'] = data[0]
#df['latitude'] = data[1]
#df['coordinates'] = df[['latitude', 'longitude']].apply(lambda x: ','.join(x), axis=1)

In [86]:
df.created_at = df.created_at.apply(lambda x: pd.to_datetime(x).date())

In [87]:
date_after = pd.to_datetime('2020-08-21') #this is where our twitter data begins
date_before = pd.to_datetime('2020-09-20') #this is where our twitter data ends
df = df[df['created_at'] >= date_after]
df = df[df['created_at'] <= date_before]

In [88]:
df['longitude'] = df['geo'].apply(lambda x: x[1])
df['latitude'] = df['geo'].apply(lambda x: x[0])
df = df[df['longitude'].between(-125, -115)]
df = df[df['latitude'].between(33,42)]
df

,created_at,full_text,geo,place,truncated,display_text_range,user,id,entities,possibly_sensitive,lang,longitude,latitude
179578,2020-08-21,"First #family #dinner, since #Covid started! #...","[34.13480313, -118.25159489]","{'id': 'fbd6d2f5a4e4a15e', 'url': 'https://api...",False,"[0, 204]","{'id': 17258158, 'id_str': '17258158', 'name':...",1296597988266172416,"{'hashtags': [{'text': 'family', 'indices': [6...",False,en,-118.251595,34.134803
179589,2020-08-21,"Bought the property in January, and the guys s...","[39.2269, -120.081]","{'id': '01fb262641c7d57a', 'url': 'https://api...",False,"[0, 235]","{'id': 81999083, 'id_str': '81999083', 'name':...",1296599312856690696,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-120.081000,39.226900
179596,2020-08-21,Quarantine has been extremely difficult on my ...,"[34.0564, -118.2445]","{'id': '3b77caf94bfc81fe', 'url': 'https://api...",False,"[0, 239]","{'id': 42839960, 'id_str': '42839960', 'name':...",1296600538407153664,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-118.244500,34.056400
179611,2020-08-21,I think 2020 is REALLY out to get us! #Covid w...,"[38.5763, -121.4927]","{'id': 'b71fac2ee9792cbe', 'url': 'https://api...",False,"[0, 155]","{'id': 31538510, 'id_str': '31538510', 'name':...",1296602845471371264,"{'hashtags': [{'text': 'Covid', 'indices': [38...",False,en,-121.492700,38.576300
179620,2020-08-21,Corona CA Thu Aug 20th PM Forecast: TONIGHT Mo...,"[33.8752, -117.5655]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",False,"[0, 89]","{'id': 354691403, 'id_str': '354691403', 'name...",1296604358134706176,"{'hashtags': [], 'symbols': [], 'user_mentions...",True,en,-117.565500,33.875200
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211700,2020-09-20,Here I am spending my weekend in isolation sta...,"[36.1675, -115.149]","{'id': '5c2b5e46ab891f07', 'url': 'https://api...",True,"[0, 221]","{'id': 59358511, 'id_str': '59358511', 'name':...",1307818416078598144,"{'hashtags': [{'text': 'chilling', 'indices': ...",False,en,-115.149000,36.167500
211704,2020-09-20,She wasn’t expecting the fam to show up. Happy...,"[33.81417, -117.50838]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",True,"[0, 134]","{'id': 1188082171, 'id_str': '1188082171', 'na...",1307819830586867712,"{'hashtags': [{'text': 'Gravenberg', 'indices'...",False,en,-117.508380,33.814170
211706,2020-09-20,"Good air bad air, feel like don’t feel like it...","[33.8753, -117.566]","{'id': '5e4b6834e36e68fa', 'url': 'https://api...",True,"[0, 209]","{'id': 912199619685244929, 'id_str': '91219961...",1307821453367275527,"{'hashtags': [{'text': 'babybeast', 'indices':...",False,en,-117.566000,33.875300
211711,2020-09-20,And this state believes they can do ballot mas...,"[36.08157349, -115.1483371]","{'id': '8fa6d7a33b83ef26', 'url': 'https://api...",True,"[0, 62]","{'id': 65988563, 'id_str': '65988563', 'name':...",1307827958116052992,"{'hashtags': [], 'symbols': [], 'user_mentions...",False,en,-115.148337,36.081573


In [89]:
geolocator = Nominatim(user_agent='katiecason@berkeley.edu')
location = df['geo'].apply(lambda x: geolocator.reverse(x))

In [90]:
states = location.apply(lambda x: x.raw['address'].get('state', ''))
counties = location.apply(lambda x: x.raw['address'].get('county', ''))

In [91]:
df['state'] = states
df['county'] = counties

In [92]:
df = df[df.state == 'California']

In [93]:
df.to_csv('../w207_finalproject/data/august-september.csv')